In [1]:
import sys
import os
import yaml
sys.path.insert(1, '..')
os.chdir('..')

from data_formatter.base import *

/Users/renatsergazinov/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load yaml config file
with open('./config/dubosson.yaml', 'r') as f:
    config = yaml.safe_load(f)

In [3]:
# initialise formatter
formatter = DataFormatter(config)

Dropped segments: 0
Extracted segments: 9


In [4]:
formatter.data.dtypes

time           datetime64[ns]
id                    float64
gl                    float64
time_year             float64
time_month            float64
time_day              float64
time_hour             float64
time_minute           float64
id_segment            float64
dtype: object

In [5]:
formatter._column_definition

[('id', <DataTypes.REAL_VALUED: 0>, <InputTypes.ID: 4>),
 ('time', <DataTypes.DATE: 2>, <InputTypes.TIME: 6>),
 ('gl', <DataTypes.REAL_VALUED: 0>, <InputTypes.TARGET: 0>),
 ('time_year', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
 ('time_month', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
 ('time_day', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
 ('time_hour', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
 ('time_minute', <DataTypes.REAL_VALUED: 0>, <InputTypes.KNOWN_INPUT: 2>),
 ('id_segment', <DataTypes.CATEGORICAL: 1>, <InputTypes.SID: 5>)]

In [6]:
formatter.train_data

,time,id,gl,time_year,time_month,time_day,time_hour,time_minute,id_segment
0,2014-10-01 19:14:00,0.0,185.4,2014.0,10.0,1.0,19.0,14.0,0.0
1,2014-10-01 19:19:00,0.0,178.2,2014.0,10.0,1.0,19.0,19.0,0.0
2,2014-10-01 19:24:00,0.0,176.4,2014.0,10.0,1.0,19.0,24.0,0.0
3,2014-10-01 19:29:00,0.0,172.8,2014.0,10.0,1.0,19.0,29.0,0.0
4,2014-10-01 19:34:00,0.0,169.2,2014.0,10.0,1.0,19.0,34.0,0.0
...,...,...,...,...,...,...,...,...,...
6793,2014-10-05 03:20:00,6.0,172.8,2014.0,10.0,4.0,9.0,25.0,7.0
6794,2014-10-05 03:25:00,6.0,172.8,2014.0,10.0,4.0,9.0,25.0,7.0
6795,2014-10-05 03:30:00,6.0,172.8,2014.0,10.0,4.0,9.0,25.0,7.0
6796,2014-10-05 03:35:00,6.0,172.8,2014.0,10.0,4.0,9.0,25.0,7.0
